# Semana 4: Implementación de Modelos de Deep learning

Autor: Dan Santivañez Gutarra


### Objetivos:
*   Explorar el uso de una Red Neuronal Artificial (ANN)
*   Explorar el uso de una Red Neuronal Convolucional (CNN)
*   Explorar el uso de una Red Neuronal Recurrente (RNN)


## 1. Configuración del Entorno
Instalamos e importamos las librerías necesarias para trabajar con los datos y modelos.


In [ ]:
!pip install kagglehub
import kagglehub
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, mean_squared_error, mean_absolute_error, precision_score, recall_score
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

## 2. Red Neuronal Artificial (ANN) - Heart Disease

**Red Neuronal Artificial (ANN):**  
Es un modelo básico con capas densamente conectadas, ideal para clasificación y regresión con datos tabulares, como predecir enfermedades cardíacas.


### 2.1 Cargar y explorar el dataset

In [ ]:
path = kagglehub.dataset_download("redwankarimsony/heart-disease-data")
heart_disease_path = os.path.join(path, 'heart_disease_uci.csv')
df_heart = pd.read_csv(heart_disease_path)
print("Primeras filas del dataset:")
df_heart.head()

### 2.2 Preprocesamiento

In [ ]:
# - Eliminamos valores nulos.
df_heart = df_heart.dropna()
le = LabelEncoder()
for column in ['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']:
    df_heart[column] = le.fit_transform(df_heart[column])

In [ ]:
# - Codificamos variables categóricas.
X = df_heart.drop(['id', 'num'], axis=1)
y = df_heart['num'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
# - Escalamos características numéricas.
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 2.3 Construcción del modelo ANN

In [ ]:
model_ann = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model_ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model_ann.summary()

In [ ]:
history_ann = model_ann.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

### 2.5 Evaluación y Visualización

In [ ]:
# Evaluación
y_pred = (model_ann.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)


In [ ]:
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


In [ ]:
# Matriz de Confusión
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matriz de Confusión - ANN')
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.show()

In [ ]:
# Pérdida y Precisión
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history_ann.history['loss'], label='Pérdida Entrenamiento')
plt.plot(history_ann.history['val_loss'], label='Pérdida Validación')
plt.title('Pérdida - ANN')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history_ann.history['accuracy'], label='Precisión Entrenamiento')
plt.plot(history_ann.history['val_accuracy'], label='Precisión Validación')
plt.title('Precisión - ANN')
plt.legend()
plt.show()

In [ ]:
# Distribución de Clases
plt.figure(figsize=(6,4))
sns.countplot(x=y)
plt.title('Distribución de Enfermedad Cardíaca')
plt.xlabel('Enfermedad (0: No, 1: Sí)')
plt.ylabel('Cantidad')
plt.show()

In [ ]:

# Curva Precision-Recall
from sklearn.metrics import precision_recall_curve
prec, rec, _ = precision_recall_curve(y_test, model_ann.predict(X_test))
plt.figure(figsize=(6,4))
plt.plot(rec, prec, marker='.')
plt.title('Curva Precision-Recall - ANN')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()


In [ ]:
# Ejemplos de Predicciones para ANN
# Seleccionamos 10 ejemplos aleatorios del conjunto de prueba
indices = np.random.choice(len(X_test), 10, replace=False)
X_sample = X_test[indices]
y_sample_real = y_test.iloc[indices] if isinstance(y_test, pd.Series) else y_test[indices]
y_sample_pred = (model_ann.predict(X_sample) > 0.5).astype(int).flatten()

# Mostramos las predicciones y las etiquetas reales
for i in range(10):
    print(f"Ejemplo {i+1}:")
    print(f"Características (escaladas): {X_sample[i][:5]}...")  # Mostramos las primeras 5 características
    print(f"Predicción: {y_sample_pred[i]}, Real: {y_sample_real[i]}")
    print(f"Correcto: {'Sí' if y_sample_pred[i] == y_sample_real[i] else 'No'}")
    print("-" * 30)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Función objetivo
def f(x):
    return x**2

# Derivada de la función objetivo
def df(x):
    return 2*x

# Parámetros del gradiente descendiente
learning_rate = 0.1
iterations = 20
x = 10  # Valor inicial

# Lista para almacenar los valores de x
x_values = [x]

# Proceso de gradiente descendiente
for i in range(iterations):
    gradient = df(x)
    x = x - learning_rate * gradient
    x_values.append(x)

# Visualización
plt.figure(figsize=(10, 6))
x_range = np.linspace(-10, 10, 400)
plt.plot(x_range, f(x_range), label='f(x) = x²')
plt.scatter(x_values, [f(x) for x in x_values], color='red', label='Pasos del Gradiente Descendiente')
for i, (x_val, y_val) in enumerate(zip(x_values, [f(x) for x in x_values])):
    plt.text(x_val, y_val, f'Iter {i}', fontsize=8)
plt.title('Gradiente Descendiente para f(x) = x²')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()
plt.show()

## 3. Red Neuronal Convolucional (CNN) - Digit Recognizer

**Red Neuronal Convolucional (CNN):**  
Diseñada para imágenes, usa capas convolucionales y pooling para extraer y reducir características.


### 3.1 Cargar y explorar el dataset

In [ ]:
from tensorflow.keras.datasets import mnist
import numpy as np

# Cargar el dataset MNIST
(X_train_full, y_train_full), (X_test, y_test) = mnist.load_data()

# Mostrar información básica
print("Dimensiones del conjunto de entrenamiento:", X_train_full.shape)
print("Dimensiones del conjunto de prueba:", X_test.shape)
print("Ejemplo de etiquetas:", y_train_full[:5])

### 3.2 Preprocesamiento

In [ ]:
from sklearn.model_selection import train_test_split

# Normalizar los valores de los píxeles al rango [0, 1]
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# Añadir dimensión para el canal (1, porque son imágenes en escala de grises)
X_train_full = X_train_full.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

# Dividir el conjunto de entrenamiento en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_val:", X_val.shape)
print("Dimensiones de X_test:", X_test.shape)

### 3.3 Construcción del modelo CNN

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model_cnn = keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model_cnn.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

model_cnn.summary()

### 3.4 Entrenamiento

In [ ]:
history_cnn = model_cnn.fit(X_train, y_train,
                            epochs=10,
                            batch_size=64,
                            validation_data=(X_val, y_val),
                            verbose=1)

### 3.5 Evaluación y Visualización

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Predicciones en el conjunto de prueba
y_pred = np.argmax(model_cnn.predict(X_test), axis=1)

In [ ]:
# Métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
cm = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

In [ ]:
# Matriz de Confusión
plt.figure(figsize=(10,8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matriz de Confusión - CNN')
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.show()

In [ ]:
# Pérdida y Precisión
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history_cnn.history['loss'], label='Pérdida Entrenamiento')
plt.plot(history_cnn.history['val_loss'], label='Pérdida Validación')
plt.title('Pérdida - CNN')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history_cnn.history['accuracy'], label='Precisión Entrenamiento')
plt.plot(history_cnn.history['val_accuracy'], label='Precisión Validación')
plt.title('Precisión - CNN')
plt.legend()
plt.show()

In [ ]:
# Ejemplos de Predicciones
fig, axes = plt.subplots(5, 5, figsize=(15,15))
axes = axes.ravel()
for i in range(25):
    axes[i].imshow(X_test[i].reshape(28,28), cmap='gray')
    axes[i].set_title(f"Pred: {y_pred[i]}\nReal: {y_test[i]}")
    axes[i].axis('off')
plt.tight_layout()
plt.show()

## 4. Red Neuronal LSTM - Stock Prices

**Red Neuronal de Memoria a Largo Plazo (LSTM):**

Ideal para datos secuenciales, como series temporales de precios de acciones.

### 4.1 Cargar y explorar el dataset

In [ ]:
path = kagglehub.dataset_download("borismarjanovic/price-volume-data-for-all-us-stocks-etfs")
stock_path = os.path.join(path, 'Stocks/aapl.us.txt')
df_stock = pd.read_csv(stock_path)
print("Primeras filas del dataset:")
print(df_stock.head())

### 4.2 Preprocesamiento

In [ ]:
data = df_stock['Close'].values.reshape(-1, 1)
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

In [ ]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

In [ ]:
seq_length = 60
X, y = create_sequences(data_scaled, seq_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### 4.3 Construcción del modelo LSTM

In [ ]:
model_lstm = keras.Sequential([
    layers.LSTM(50, return_sequences=True, input_shape=(seq_length, 1)),
    layers.LSTM(50),
    layers.Dense(1)
])
model_lstm.compile(optimizer='adam', loss='mse')

model_lstm.summary()

### 4.4 Entrenamiento

In [ ]:
history_lstm = model_lstm.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

### 4.5 Evaluación y Visualización

In [ ]:
# Evaluación
y_pred = model_lstm.predict(X_test)
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

In [ ]:
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MAPE: {mape:.4f}%")

(MSE: 0.6853, RMSE: 0.8279, MAE: 0.4284, MAPE: 11.3580%) indican que el modelo LSTM tiene un rendimiento moderado para predecir precios de acciones. El error promedio absoluto es de 0.4284 unidades y el error porcentual es de 11.36%, lo cual sugiere que las predicciones son razonables pero no perfectas.



In [ ]:
# Ejemplos de Predicciones para LSTM
# Seleccionamos un subconjunto de 100 puntos para visualizar
n_points = 100
plt.figure(figsize=(12, 6))
plt.plot(y_test[:n_points], label='Valores Reales', color='blue')
plt.plot(y_pred[:n_points], label='Predicciones', color='red', linestyle='--')
plt.title('Predicciones vs. Valores Reales - LSTM (Primeros 100 puntos)')
plt.xlabel('Índice')
plt.ylabel('Precio de Cierre (Normalizado)')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de Dispersión
# Muestra la relación entre valores reales y predichos en un gráfico de dispersión.
# Los puntos cercanos a la línea roja (y=x) indican predicciones precisas.
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Valores Reales')
plt.ylabel('Predicciones')
plt.title('Predicciones vs. Valores Reales - LSTM')
plt.show()

In [ ]:
# Histograma de Errores
# Muestra la distribución de los errores (diferencia entre valores reales y predichos).
# La curva KDE indica la densidad; un pico cerca de 0 sugiere predicciones precisas.
errors = y_test - y_pred
plt.figure(figsize=(8,6))
sns.histplot(errors, bins=50, kde=True)
plt.title('Distribución de Errores - LSTM')
plt.xlabel('Error')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Pérdida
plt.figure(figsize=(8,4))
plt.plot(history_lstm.history['loss'], label='Pérdida Entrenamiento')
plt.plot(history_lstm.history['val_loss'], label='Pérdida Validación')
plt.title('Pérdida - LSTM')
plt.legend()
plt.show()

In [ ]:
# Residuos
# Grafica los errores (residuos) frente al índice de las predicciones.
# Puntos cerca de la línea roja (y=0) indican predicciones precisas.
plt.figure(figsize=(8,6))
plt.plot(errors, 'o', alpha=0.5)
plt.axhline(0, color='r', linestyle='--')
plt.title('Residuos - LSTM')
plt.xlabel('Índice')
plt.ylabel('Error')
plt.show()

## 5. Conclusiones
- ANN: Clasificación binaria con métricas detalladas y visualizaciones.
- CNN: Clasificación multiclase con ejemplos y distribución de clases.
- LSTM: Predicción de series temporales con análisis de errores extendido.